<a href="https://colab.research.google.com/github/g3ferreira/DOC-JavaPOS/blob/master/missing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import requests
#import urllib3
#from contextlib import closing
import json
import cv2
import requests
#import os
import datetime
import time 
#import datetime

#ipServer = "localhost"


class Jsoncontroller(object):
    def __init__(self):
        self.ipServer = "localhost"

    def getSTRElapsedTime(self, initime, endtime):
        times = []         
        elapsed_time = endtime  - initime
        totalSecs, sec = divmod(elapsed_time, 60)
        hr, min = divmod(totalSecs, 60)
        elapsed_time = "%02d:%02d:%02d" % (hr, min, sec)   
        #print(str(elapsed_time))
    
        string_inittime = str(time.strftime("%H:%M:%S", time.localtime(initime)))
        string_endtime = str(time.strftime("%H:%M:%S", time.localtime(endtime)))
        #string_elapsed = str(time.strftime("%H:%M:%S", elapsed_time))
        #times.append(inittacttime)
        times.append(string_inittime)
        times.append(string_endtime)
        times.append(elapsed_time)
        #print(string_inittime)
        #print(string_endtime)
        #print(string_elapsed)
        return times


    def request_setup_model(self, c_device_model, c_camera_distance, coordenadas):
        coordenadas_split = ""
        for coordenada in coordenadas: 
            coordenadas_split = coordenadas_split+coordenada.coord_roi_x1 + "," + coordenada.coord_roi_x2 + "," +  coordenada.coord_roi_y1 + "," + coordenada.coord_roi_y2 +  "#"

        print("Coordenadas: " + coordenadas_split)    
        
        fields = {        
            'c_device_model': c_device_model, 
            'c_qt_box': int(len(coordenadas)), 
            'c_camera_distance': c_camera_distance, 
            'c_positions': coordenadas_split
        }
        response = requests.post("http://"+self.ipServer+":3700/config_app", data = fields)
    
        r = json.loads(response.text)
        print(response.text)
        print(response.status_code)

        return 


    def request_barcode_read(self,device_model, qt_box, test_start_time, test_end_time, test_items, test_result, image):
        elapsed_time = test_end_time  - test_start_time
        totalSecs, sec = divmod(elapsed_time, 60)
        hr, min = divmod(totalSecs, 60)
        test_tacttime =  "%02d:%02d:%02d" % (hr, min, sec) 
        times = self.getSTRElapsedTime(test_start_time, test_end_time)
        
        #inittacttime = str(time.strftime("%H:%M:%S", time.localtime(inittacttime)))
        cv2.imwrite('barcode_read.jpg', image)
        multipart_form_data = {
            'image': ('barcode_read.jpg', open('barcode_read.jpg', 'rb'))
        }
        
        test_date = str(datetime.datetime.now().strftime("%Y-%m-%d"))
        fields = {
            'device_model': device_model,
            'qt_box': qt_box,
            'test_date': str(test_date),
            'test_start_time': times[0],
            'test_end_time': times[1],
            'test_time': times[2],      
            'test_items': test_items,
            'test_name': "Leitura de Código de barras",
            'test_result': test_result
            #'test_tacttime': test_tacttime
        }
        response = requests.post("http://"+self.ipServer+":3700/barcode_test", data = fields, files=multipart_form_data)
    
        r = json.loads(response.text)
        id_test =  str(r.get('docs')[0]['_id'])     #Modificado essa linha
        print("id_test: "+id_test)
        print(response.text)
        print(response.status_code)

        return id_test


    def request_lacre_test(self, id_test, start_time, end_time, qt_items, test_result, image): 
        #image_filename = os.path.basename(image_path)
        #print(image_filename)
        elapsed_time = end_time  - start_time
        totalSecs, sec = divmod(elapsed_time, 60)
        hr, min = divmod(totalSecs, 60)
        times = self.getSTRElapsedTime(start_time, end_time)
        #inittacttime = str(time.strftime("%H:%M:%S", time.localtime(inittacttime)))
        cv2.imwrite('lacre.jpg', image)
        multipart_form_data = {
            'image': ('lacre.jpg', open('lacre.jpg', 'rb'))
        }
        fields = {
            'id_test': id_test,
            'test_date': str(datetime.datetime.now().strftime("%Y-%m-%d")),
            'test_start_time': times[0],
            'test_end_time': times[1],
            'test_time': times[2],      
            'qt_items': qt_items,
            'test_name': "Inspeção de Lacre",
            'test_result': test_result
            #'test_tacttime': test_tacttime
        }
        response = requests.post("http://"+self.ipServer+":3700/lacre_test", data = fields, files=multipart_form_data)
    
        
        r = json.loads(response.text)
        id_test =  str(r.get('docs')[0]['lacre_test'][0]['_id'])     #Modificado essa linha
        print("id_lacre: "+id_test)
        print(response.text)
        #print(response.status_code)
        return id_test  
        

    def request_label_test(self, id_test, start_time, end_time, test_time_oper, test_total_time, test_items, qt_items, test_result, barcode_masterbox, image): 
        #image_filename = os.path.basename(image_path)
        #print(image_filename)
        #elapsed_time = end_time  - start_time
        #totalSecs, sec = divmod(elapsed_time, 60)
        #hr, min = divmod(totalSecs, 60)
        times = self.getSTRElapsedTime(start_time, end_time)
        #inittacttime = str(time.strftime("%H:%M:%S", time.localtime(inittacttime)))
        cv2.imwrite('lacre.jpg', image)
        multipart_form_data = {
            'image': ('lacre.jpg', open('lacre.jpg', 'rb'))
        }
        fields = {
            'id_test': id_test,
            'test_date': str(datetime.datetime.now().strftime("%Y-%m-%d")),
            'test_start_time': times[0],
            'test_end_time': times[1],
            'test_time': times[2],    
            'test_time_oper': test_time_oper,  
            'test_total_time': test_total_time,            
            'test_items': test_items,
            'qt_items': qt_items,
            'test_name': "Inspeção de Label",
            'barcode_masterbox': barcode_masterbox,
            'test_result': test_result
            #'test_tacttime': test_tacttime
        }
        response = requests.post("http://"+self.ipServer+":3700/label_test", data = fields, files=multipart_form_data)
    
        
        r = json.loads(response.text)
        #id_test =  str(r.get('docs')[0]['label_test'][0]['_id'])     #Modificado essa linha
        #print("id_label: "+id_test)
        #print(response.text)
        #print(response.status_code)
        return #id_test      
        
        

    def request_config_get_models(self):      
        response = requests.get("http://"+self.ipServer+":3700/config_app")
        r = json.loads(response.text)
        models =  str(r.get("docs"))
        print("models:" + models)
        print(response.text)
        # print(response.status_code)
        return models         

    def request_config_get_model_by_id(self, modelo, quantidade):      
        response = requests.get("http://"+self.ipServer+":3700/config_app/" + modelo + "/" + quantidade)
        r = json.loads(response.text)
        #str(r.get('docs')[0]['label_test'][0]['_id'])
        model =  str(r.get("docs"))
        print("model:" + model)
        # print(response.status_code)
        return model        

    def request_config_get_positions_by_id(self, modelo, quantidade):      
        response = requests.get("http://"+self.ipServer+":3700/config_app/" + modelo + "/" + quantidade)
        r = json.loads(response.text)
        #str(r.get('docs')[0]['label_test'][0]['_id'])
        positions =  str(r.get("docs")['positions']).split("}")
        
  
        #print(positions)
        print(positions)
        # print(response.status_code)
        return positions        


    def request_config_status_teste(self):      
        response = requests.get("http://"+self.ipServer+":3700/model_status")
        r = json.loads(response.text)
        #str(r.get('docs')[0]['label_test'][0]['_id'])
        status =  str(r.get("model_status"))#['model_status'])
        
  
        #print(positions)
        #print(status)
        # print(response.status_code)
        return status    



# jsoncontroller       

In [ ]:

import sys

sys.path.insert(0, 'D:/genilson-ferreira/projetos/missing-label')
sys.path.insert(0, 'D:/genilson-ferreira/projetos/missing-label/controllers')
from imutils.video import VideoStream
from pyzbar import pyzbar
import argparse
import datetime
import imutils
import time
import cv2
import winsound
import requests
import json
from jsoncontroller import Jsoncontroller
from collections import namedtuple
import numpy as np

class Coordenada:
      def __init__(self):            
            self.coord_roi_x1 = None
            self.coord_roi_x2 = None
            self.coord_roi_y1 = None
            self.coord_roi_y2 = None
            

json_controller = Jsoncontroller()            
def getBarcode(image_roi):
        barcodes = set()
        frame = image_roi
        barcodes = pyzbar.decode(frame)
        count_imei = 0
        barcode_resultado = []
        barcode_masterbox = ""
        for barcode in barcodes:            
            barcodeData = barcode.data.decode("utf-8")
            if(barcode.type == "CODE128" and len(barcodeData) == 9):
                barcode_masterbox = barcodeData
            elif(barcode.type == "CODE128" and len(barcodeData) == 15):
                count_imei = count_imei + 1
                text = "IMEI: {}".format(barcodeData)
                barcode_resultado.append(barcodeData)
                (x, y, w, h) = barcode.rect
                margin_ly = 90
                margin_ty = 260
                margin_x = 270
                if(y > 400 and y < 600):  
                      cv2.line(frame,(x-20,y + 255),(x + margin_x, y + 255),(188,0,4),40)   #LINHA IMEI   
                      cv2.putText(frame, text, (x, y + margin_ty),  cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)                
                      cv2.line(frame,(x,y),(x, y + 220),(0,255,0),5)   #LINHA BARCODE             
                elif(y > 1000 and y < 1500):     
                      cv2.line(frame,(x-20,y + 255),(x + margin_x, y + 255),(188,0,4),40)     #LINHA IMEI  
                      cv2.putText(frame, text, (x, y + margin_ty),  cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255 , 255, 255), 2)          
                      cv2.line(frame,(x,y),(x, y + 220),(0,255,0),5)   #LINHA BARCODE  
                else:
                      margin_y = 80
                      cv2.line(frame,(x, y + margin_y),(x + margin_x, y + margin_y),(188,0,4),40)     #LINHA IMEI  
                      cv2.putText(frame, text, (x, y + margin_ly),  cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)    
                      cv2.line(frame,(x,y),(x, y - 220),(0,255,0),5)          #LINHA BARCODE  
                #cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, '', (x, y - 10),  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                #cv2.putText(image, '', Point(0, img.cols/2),  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                text = "{} x: {} y:{} w:{} h:{}".format(text, x,y,w,h)
                #print(text)    
                cv2.imwrite("result.jpg", frame)                
       
        #dsize = (1920, 1080)
        #resize image
            
        #frame = cv2.resize(frame, dsize)
        cv2.line(frame,(70,70),(70 + 400, 70),(0,0,0),70)   #LINHA BARCODE       
        cv2.putText(frame, "barcodes: " + str(count_imei), (70, 70 ),  cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
        return (frame, barcode_resultado, barcode_masterbox)

def get_positions(modelo, quantidade):
    global json_controller
    positions =  json_controller.request_config_get_positions_by_id(modelo, str(quantidade))
    positions.pop(len(positions)-1)
    positions.pop(len(positions)-1)
    
    coordenadas = []
    
    for position in positions:
        coordenada =  Coordenada()
        position_split = position.split(",")
        if(len(position_split) == 5):
        #while(len(position_split) !=4):
            position_split.pop(len(position_split)-1)  
        elif(len(position_split) == 6 ):
            position_split.pop(0)  
            position_split.pop(len(position_split)-1)  
        
        position = position_split              
        coordenada.coord_roi_x1 = position[0].split(":")[1].strip()#.replace("'", '')         
        coordenada.coord_roi_x2 = position[1].split(":")[1].strip()#.replace("'", '')     
        coordenada.coord_roi_y1 = position[2].split(":")[1].strip()#.replace("'", '')     
        coordenada.coord_roi_y2 = position[3].split(":")[1].strip()#.replace("'", '')     
        coordenadas.append(coordenada)
        
    return coordenadas
    
def desenhar_coordenadas(coordenadas, image_nova):
    global image
    posicao = 1
    
    green = (0,255,0)
    red = (0,0,255)
    quantidade_lacre_ok = 0
    resultado_teste = True
    for coordenada in coordenadas:        
        image_roi  = image_nova[int(coordenada.coord_roi_y1):int(coordenada.coord_roi_y2), int(coordenada.coord_roi_x1):int(coordenada.coord_roi_x2)]
        resultado = identificao_lacre(image_roi)
        if(resultado):           
        #cv2.imshow('roi', img)
        #cv2.imshow('roi2', gaussian)
        #cv2.waitKey(0)
            quantidade_lacre_ok = quantidade_lacre_ok + 1
            cv2.rectangle(image_nova,(int(coordenada.coord_roi_x1), int(coordenada.coord_roi_y2) ),(int(coordenada.coord_roi_x2), int(coordenada.coord_roi_y1)),green,3) 
        else: 
            cv2.rectangle(image_nova,(int(coordenada.coord_roi_x1), int(coordenada.coord_roi_y2) ),(int(coordenada.coord_roi_x2), int(coordenada.coord_roi_y1)),red,3) 
            resultado_teste = False
        cv2.putText(image_nova, str(posicao), ((int(coordenada.coord_roi_x1),  int(coordenada.coord_roi_y2)- 10)),  cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 4)
        posicao = posicao + 1
        
        
        #image = image_nova 
    return (image_nova,resultado_teste, quantidade_lacre_ok)    

def identificao_lacre(image_roi): 
    resultado = True 
    gray  = cv2.cvtColor(image_roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray,5)
    gaussian = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)
    contours, hierarchy = cv2.findContours(gaussian, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    img = cv2.drawContours(image_roi, contours, -1, (0,255,0), 1)
    print(len(contours))
    if(len(contours) < 100):
        resultado = False    
    
    return resultado 

def comparando_barcodes(lista_barcode_origem, lista_barcode_label):
    lista_barcodes_nao_encontrados = []
    resultado = True
    for barcode in lista_barcode_origem:
    
        if not(any(barcode in s for s in lista_barcode_label)): 
            lista_barcodes_nao_encontrados.append(barcode)
            resultado = False
            
    return(resultado, lista_barcodes_nao_encontrados)
'''
image = cv2.imread('imagens/lacre_2.jpg')   
image_original =  cv2.imread('imagens/lacre_2.jpg')   

a =  cv2.imread('imagens/lacre_2.jpg')      
b =  cv2.imread('imagens/lacre_2.jpg')      
#image = cv2.imread('imagens/lacre_1.jpg')
cv2.namedWindow('image', cv2.WINDOW_GUI_NORMAL)

while(1):
   
    k = cv2.waitKey(1) & 0xFF
    if k == ord('m'):
        mode = not mode
    
    if k == ord('c'):        
        test_start_total_time = time.time()
        test_start_time = time.time()
        device_model = "A52"
        qt_models = 10
        
        resultados = getBarcode(image)
        image = resultados[0]
        barcodes = resultados[1]
        barcode_string = ""
        cont = 0
        for barcode in barcodes:
            cont = cont + 1
            if(cont != len(barcodes)):
                 barcode_string = barcode_string + barcode + "#"
            else:
                 barcode_string = barcode_string + barcode
        test_end_time = time.time()
        test_items = barcode_string # "012345678901t#222222222222a#33333333333s#555555555555d#666666666666f#77777777777g#88888888888888h#999999999j#1000000k#111111111111b"
        qt_items = str(len(barcodes))
        #buscar quantidade de celular registrada
        if(qt_items != qt_models):
            test_result = "OK"
        else:
            test_result = "NG"
        
        test_end_time = time.time()     
        id_test = json_controller.request_barcode_read(device_model, qt_items, test_start_time, test_end_time, test_items, test_result, image)
        print("id_test: " + str(id_test))
        time.sleep(2)
        test_start_time = time.time()
        coordenadas = get_positions(device_model, 10)    
        resultado_lacre = desenhar_coordenadas(coordenadas, image)   
        image = resultado_lacre[0]
        resultado_teste_lacre = resultado_lacre[1]
        qt_items = resultado_lacre[2]
        if(resultado_teste_lacre):
            test_result = "OK"
        else:
            test_result = "NG"
        
        end_time = time.time()    
        id_lacre = json_controller.request_lacre_test(id_test, test_start_time, end_time, qt_items, test_result, image)
      
    if k == ord('l'):
        print("testar lacre")
        coordenadas =get_positions("A52", 10)    
        image = desenhar_coordenadas(coordenadas, image_original)   
        
    if k == ord('3'):
        print("testar lacre")
        coordenadas =get_positions("A52", 3)    
        image = desenhar_coordenadas(coordenadas, a)   
        
    if k == ord('7'):
        print("testar lacre")
        coordenadas =get_positions("A52", 7)    
        image = desenhar_coordenadas(coordenadas, b)   
        
    
    elif k == 27:
        break
    
    cv2.imshow('image',image)        
cv2.destroyAllWindows()
'''
def get_status_teste():
    
    while(True):
        status = json_controller.request_config_status_teste()
        if(status == "1"):
            image_origem = cv2.imread('imagens/lacre_1.jpg') 
            test_start_total_time = time.time()
            test_start_time = time.time()
            device_model = "A52"
            qt_models = 10
            
            resultados = getBarcode(image_origem)
            image = resultados[0]
            barcodes = resultados[1]
            barcode_string = ""
            cont = 0
            for barcode in barcodes:
                cont = cont + 1
                if(cont != len(barcodes)):
                    barcode_string = barcode_string + barcode + "#"
                else:
                    barcode_string = barcode_string + barcode
            test_end_time = time.time()
            test_items = barcode_string # "012345678901t#222222222222a#33333333333s#555555555555d#666666666666f#77777777777g#88888888888888h#999999999j#1000000k#111111111111b"
            qt_items = str(len(barcodes))
            #buscar quantidade de celular registrada
            if(qt_items != qt_models):
                test_result = "OK"
            else:
                test_result = "NG"
            
            test_end_time = time.time()     
            id_test = json_controller.request_barcode_read(device_model, qt_items, test_start_time, test_end_time, test_items, test_result, image)
            print("id_test: " + str(id_test))
            time.sleep(2)
            test_start_time = time.time()
            coordenadas = get_positions(device_model, 10)    
            resultado_lacre = desenhar_coordenadas(coordenadas, image)   
            image = resultado_lacre[0]
            resultado_teste_lacre = resultado_lacre[1]
            qt_items = resultado_lacre[2]
            if(resultado_teste_lacre):
                test_result = "OK"
            else:
                test_result = "NG"
            
            end_time = time.time()    
            id_lacre = json_controller.request_lacre_test(id_test, test_start_time, end_time, qt_items, test_result, image)
            start_time_oper = time.time()
            time.sleep(4)
            end_time_oper = time.time()
            test_time_oper = json_controller.getSTRElapsedTime(start_time_oper, end_time_oper)
            print(test_time_oper)
            
            test_start_time = time.time()
            image_label= cv2.imread('imagens/label1.jpg')   
            resultado_label = getBarcode(image_label)
            image_label = resultado_label[0]
            resultado_match_barcodes = comparando_barcodes(barcodes, resultado_label[1])  
            codigo_barra_masterbox = resultado_label[2]
            if(resultado_match_barcodes[0]):
                test_result = "OK"
                qt_imeis = len(barcodes)
            else:
                test_result = "NG"
                cont = 0
                barcode_string = ""
                for barcode in resultado_match_barcodes[1]:
                    cont = cont + 1
                    if(cont != len(resultado_match_barcodes[1])):
                        barcode_string = barcode_string + barcode + "#"
                    else:
                        barcode_string = barcode_string + barcode
                test_items = barcode_string
                qt_imeis = len(resultado_match_barcodes[1])
                
            end_time = time.time()             
            test_total_time = json_controller.getSTRElapsedTime(test_start_total_time, end_time)  
            id_label = json_controller.request_label_test(id_test, test_start_time, end_time, test_time_oper[2], test_total_time[2], test_items, qt_imeis, test_result, codigo_barra_masterbox, image_label)
    
            print(status)
            time.sleep(1)
        
get_status_teste()        
 #t = threading.Thread(target=worker)
 #   threads.append(t)
 #   t.start()
 
#cv2.namedWindow('image', cv2.WINDOW_GUI_NORMAL)
#image_origem = cv2.imread('imagens/label1.jpg') 
#result = getBarcode(image_origem)
#cv2.imshow("image", result[0])
#cv2.waitKey(0) 
'''
#image = cv2.imread('imagens/new/7.jpg')
image = cv2.imread('imagens/5.jpg')
dsize = (1920, 1080)

# resize image
#image = cv2.resize(image, dsize)

# load the image, clone it, and setup the mouse callback function
#image = cv2.imread(args["image"])
coordenadas = []
clone = image.copy()

result = getBarcode(image)
cv2.imshow("image", result)
cv2.waitKey(0) 
'''
   
#cv2.setMouseCallback("image", click_and_crop)
# keep looping until the 'q' key is pressed


'''
#cap = cv2.VideoCapture(1)
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FOURCC, fourcc)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
cap.set(cv2.CAP_PROP_FPS, 60)
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    #print(frame.shape())
    # Our operations on the frame come here
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    result = getBarcode(frame)
    cv2.imwrite("result.jpg", result)    
    data = {
        'e_nome' : 
            "Teste 1"
    }

    headers = {'Content-type': 'multipart/form-data'}

    files = {'e_image': open('result.jpg', 'rb')}

    r = requests.post("http://192.168.1.100:3700/test_photo", files=files, data=data)
    print(r.json())
    
    
    # Display the resulting frame
    cv2.imshow('frame',result)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
'''

#missing label

In [ ]:

import sys

sys.path.insert(0, 'D:/genilson-ferreira/projetos/missing-label')
sys.path.insert(0, 'D:/genilson-ferreira/projetos/missing-label/controllers')

import cv2
import requests
import tkinter as tk
from tkinter import simpledialog
from jsoncontroller import Jsoncontroller

drawing = False 
mode = True 
ix,iy = -1,-1
coordenadas = []
modelo = None
distancia_camera = "300"
class Coordenada:
      def __init__(self):            
            self.coord_roi_x1 = None
            self.coord_roi_x2 = None
            self.coord_roi_y1 = None
            self.coord_roi_y2 = None
            
def click_and_crop(event, x , y, flags, param):
    global ix,iy,drawing,mode, coordenadas, modelo
    '''
    if event == cv2.EVENT_MOUSEMOVE:
        
          refPt_d.append((x, y))
          cropping = True
          if len(refPt_d) == 2:
             cv2.rectangle(image, refPt_d[0], refPt_d[1], (0, 255, 0), 2)
             cv2.imshow("image", image)
    ''' 
                  
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix,iy = x,y
        cv2.rectangle(image,(ix,iy),(x,y),(0,255,0),2)
        
    #elif event == cv2.EVENT_MOUSEMOVE:
    #    if drawing == True:
    #         if mode == True:
    #             cv2.rectangle(img,(ix,iy),(x,y),(0,255,0),-2)
            #else:
            #    cv2.circle(img,(x,y),5,(0,0,255),-1)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        if mode == True:
            cv2.rectangle(image,(ix,iy),(x,y),(0,255,0),2)            
            y1 =  y
            y2 =  iy
            x1 = ix
            x2 = x                        
            coordenada  = Coordenada() 
            #print("x1: " + str(x1))
            #print("x2: " + str(x2))
            # print("y1: " + str(y1))
            # print("y2: " + str(y2))
            
        
            cv2.rectangle(image,(x1,y2),(x2,y1 ),(255,0,0),3)
            coordenadas.append(coordenada)
            cv2.putText(image, str(len(coordenadas)), (x1, y2- 10),  cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 2)
            coordenada.coord_roi_x1 = str(x1)
            coordenada.coord_roi_x2 = str(x2)
            coordenada.coord_roi_y1 = str(y1)
            coordenada.coord_roi_y2 = str(y2)
            
            #print("quantidade de lacres: " + str(len(coordenadas)))
            #image_roi  = image[y1:y2, x1:x2]

            
'''
ROOT = tk.Tk()
ROOT.withdraw()
modelo = simpledialog.askstring(title="MISSING LABEL",  prompt="DIGITE O MODELO DO CELULAR")
print("Modelo: ", modelo)
'''
modelo = "A52"
jsoncontroller_ = Jsoncontroller()

image = cv2.imread('imagens/lacre_2.jpg')        
cv2.namedWindow('image', cv2.WINDOW_GUI_NORMAL)
cv2.setMouseCallback('image',click_and_crop)

while(1):
    cv2.imshow('image',image)
    k = cv2.waitKey(1) & 0xFF
    if k == ord('m'):
        mode = not mode
    if k == ord('q'):
        mode = not mode   
    if k == ord('s'):
        print("salvando... configuração")
        jsoncontroller_.request_setup_model(modelo, distancia_camera, coordenadas)
    
    if k == ord('t'):
        print("get_models")
        jsoncontroller_.request_config_get_model_by_id(modelo, str(2))    
        #break   
    elif k == 27:
        break

cv2.destroyAllWindows()            



'''
#cap = cv2.VideoCapture(1)
#fourcc = cv2.VideoWriter_fourcc('M','J','P','G')

cap = cv2.VideoCapture(1)
cv2.namedWindow('image', cv2.WINDOW_GUI_NORMAL)
cv2.setMouseCallback('image',click_and_crop)
#cap.set(cv2.CAP_PROP_FOURCC, fourcc)
#cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
#cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
#cap.set(cv2.CAP_PROP_FPS, 60)
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    image = frame.copy()
    #print(frame.shape())
    # Our operations on the frame come here
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #result = getBarcode(frame)
    #cv2.imwrite("result_lacre.jpg", frame)      
    cv2.imshow('image',image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
'''
#lacre-setup